# Interpretability via SHAP Values 

CPH 200C Problem 2

Questions? Ask Professor Irene Chen (iychen@berkeley.edu)

As machine learning models become more inscrutable, we are interested in methods to better understand why our models make the decisions they do --- and as a result whether we can trust them. To address this problem, we typically apply either post-hoc explanations to trained models or restrict our models to specific classes that are deemed more easily understandable. The goal of this section of the problem set is to familiarize you with the SHAP values package to apply interpretability methods to the diabetes dataset we have already explored.


## 0. Load packages

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

from scipy.stats import zscore
from sklearn.preprocessing import normalize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

import xgboost as xgb
import seaborn as sns

from matplotlib import pyplot as plt

import xgboost
import shap

# TODO: change to location of unzipped csv files
fname_adult_icu   = None
fname_adult_notes = None

In [ ]:
adult_df    = pd.read_csv(fname_adult_icu)
notes_df    = pd.read_csv(fname_adult_notes)

## 1. Tabular Data From Clinical Data

Below please find some simple data cleaning. You may substitute if you prefer your own data cleaning pipeline

In [ ]:
excl_col = set(['subject_id', 'hadm_id', 'icustay_id', 'mort_oneyr', 'mort_hosp',
                'adult_icu', 'train', 'mort_icu', 'valid'])
binary_col = set(['first_hosp_stay', 'first_icu_stay', 'adult_icu',
       'eth_asian', 'eth_black', 'eth_hispanic', 'eth_other', 'eth_white',
       'admType_ELECTIVE', 'admType_EMERGENCY', 'admType_NEWBORN',
       'admType_URGENT'])
target_col = 'mort_icu'
zscore_col = [i for i in adult_df.columns if i not in excl_col and i not in binary_col]

feature_col = [i for i in adult_df.columns if i not in excl_col]

# We want to zscore the non-binary features
adult_df[feature_col] = adult_df[feature_col].apply(zscore)


Here we are training an XGBoost model to predict ICU mortality from the tabular MIMIC-III data.

In [ ]:
# Create a train/test split
is_train = adult_df['train'] == 1
is_test = adult_df['train'] == 0

X = adult_df[feature_col]
X_train = adult_df[is_train][feature_col]
y_train = adult_df[is_train][target_col]

X_test = adult_df[is_test][feature_col]
y_test = adult_df[is_test][target_col]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
d_train = xgboost.DMatrix(X_train, label=y_train)
d_test = xgboost.DMatrix(X_test, label=y_test)

# Define parameters for the model
params = {
    "eta": 0.01,
    "objective": "binary:logistic",
    "subsample": 0.5,
    "base_score": np.mean(y_train),
    "eval_metric": "logloss",
}

# Train the model
model = xgboost.train(
    params,
    d_train,
    5000,
    evals=[(d_test, "test")],
    verbose_eval=100,
    early_stopping_rounds=20,
)


In [ ]:
# Import accuracy_score from sklearn
from sklearn.metrics import accuracy_score

# Make predictions
y_pred_prob = model.predict(d_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

X_display = X[feature_col]
# Initialize JS visualization
shap.initjs()

# Explain the model's predictions using SHAP TreeExplainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# TODO: Visualize the SHAP values for data point 100 using a force_plot

In [ ]:
# TODO: Summarize the SHAP values with a summary plot 


## 2. Text Data From Clinical Notes

That was an example of how to use SHAP values to understand predictions from tabular data. Now let's explore how to visualize through clinical text.

In [ ]:
feature_col = 'chartext'
target_col = 'mort_oneyr'

ctxt = notes_df[feature_col]
vec = TfidfVectorizer(max_features=500,stop_words='english')

vec = vec.fit(ctxt)

is_train = notes_df['train'] == 1
is_test = notes_df['train'] == 0

X_train = normalize(vec.transform(notes_df[is_train][feature_col]), norm='l1', axis=0)
y_train = notes_df[is_train][target_col]

X_test = normalize(vec.transform(notes_df[is_test][feature_col]), norm='l1', axis=0)
y_test = notes_df[is_test][target_col]


In [ ]:
# SHAP 
model = LinearSVC().fit(X_train, y_train)

# Create explainer
explainer = shap.LinearExplainer(model, X_train)

# Generate SHAP values
text_to_explain = "Pt is worsening with liver failure, will transfer to surgery"
x = vec.transform([text_to_explain])

# Convert sparse matrix to dense array
x_dense = x.toarray()

# Generate SHAP values for the dense array
shap_values = explainer.shap_values(x_dense)

# TODO: Visualize the text_to_explain using a force plot

Because of the TFIDF vectorization, the string names aren't obvious. Use this dictionary to plug in feature numbers.

In [ ]:
feature_num_name_dict = {i:j for i,j in enumerate(vec.get_feature_names_out())}

# example: feature 200
print(feature_num_name_dict[200])

# TODO: What are the features increasing the likelihood of ICU mortality in the text_to_explain string?

# TODO: What are the features decreasing the likelihood of ICU mortality in the text_to_explain string?